In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd 

CODIGO PARA PODER RECUPERAR LOS DATOS

In [2]:
# Datos de conexión
server = 'practicasai.database.windows.net'
database = 'adminibm'
username = 'profesor01'
password = 'Profesor!123'
driver = '{ODBC Driver 18 for SQL Server}'

In [3]:
# Crear conexión
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

In [4]:
df = pd.read_sql_query("SELECT TOP 1000 * FROM FactInventory", conn)

C:\Users\erick\AppData\Local\Temp\ipykernel_18188\2702481104.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT TOP 1000 * FROM FactInventory", conn)


In [5]:
df

,InventoryKey,DateKey,StoreKey,ProductKey,CurrencyKey,OnHandQuantity,OnOrderQuantity,SafetyStockQuantity,UnitCost,DaysInStock,MinDayInStock,MaxDayInStock,Aging
0,1,2008-02-09,308,2086,7,16,3,9,284.1761,12,26,109,7
1,2,2007-06-09,239,643,7,21,1,9,54.7324,24,21,95,7
2,3,2009-01-31,205,18,7,17,1,6,35.6056,48,57,94,7
3,4,2007-06-02,199,1587,7,21,1,90,5.8239,59,15,118,7
4,5,2009-11-21,29,2269,7,45,13,12,10.7676,24,54,60,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2007-06-09,110,684,7,12,0,3,51.8169,67,15,117,7
996,997,2007-09-22,16,1192,7,12,0,3,194.3099,47,31,90,7
997,998,2007-09-08,291,2517,7,44,0,36,1.2042,25,30,89,7
998,999,2007-02-03,109,2185,7,5,1,3,58.5211,18,53,81,5
